In [1]:
import weaviate
import os
from weaviate import classes as wvc
client = weaviate.connect_to_local()

In [6]:
!ollama pull nomic-embed-text:latest
!ollama pull deepseek-r1:1.5b

/opt/homebrew/Cellar/python@3.12/3.12.8/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=74488) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling aabd4debf0c8... 100% ▕████████████████▏ 1.1 GB                         
pulling 369ca498f347... 100% ▕████████████████▏  387 B                         
pulling 6e4c38e1172f... 100% ▕████████████████▏ 1.1 KB                         
pulling f4d24e9138dd... 100% ▕████████████████▏  148 B                         
pulling a85fe2a2e58e... 100% ▕██████████████

In [ ]:
from weaviate import classes as wvc
#client.collections.delete("Index")
client.collections.create(
    "Index",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_ollama(
        api_endpoint="http://host.docker.internal:11434",
        model="nomic-embed-text:latest",
    ),
    generative_config=wvc.config.Configure.Generative.ollama(
        model="deepseek-r1:1.5b",
        api_endpoint="http://host.docker.internal:11434",
    )
)

In [3]:
# read csv file
import pandas as pd
from weaviate.util import generate_uuid5
collection = client.collections.get("Index")
df = pd.read_csv("trabalhos_60700009.csv")
df.columns = map(str.lower, df.columns)
# read only 10 objects
df = df.head(500)
with collection.batch.dynamic() as batch:
    for index, row in df.iterrows():
        batch.add_object(
            dict(row),
            uuid=generate_uuid5(row["indice"]),
        )
if collection.batch.failed_objects:
    print(collection.batch.failed_objects)
else:
    print("Imported with success")

Imported with success


In [7]:
collection.aggregate.over_all()

AggregateReturn(properties={}, total_count=500)

In [14]:
query = collection.query.near_text(
    query="Quais os projetos relacionados a ontologia e Organização do conhecimento?",
    return_metadata=wvc.query.MetadataQuery(distance=True),
    #grouped_task="Sumarize os conteúdos e explique porque cada um está próximo da busca: Quais os projetos relacionados a ontologia e Organização do conhecimento?",
    limit=200,
    auto_limit=1
)
for o in query.objects:
    print("###"*10 + str(o.metadata.distance))
    print("RESUMO", o.properties.get("ds_resumo"))
    print("NOME PRODUCAO", o.properties.get("nm_producao"))

# print("------"* 10)
# print(query.generated)


##############################0.31778669357299805
RESUMO A RADIOFONIA, A PRINCÍPIO UTILIZADA PARA FINS MILITARES, APENAS AO TÉRMINO DA PRIMEIRA GUERRA MUNDIAL GANHOU REPERCUSSÃO NA SOCIEDADE. IDEALIZADA NO BRASIL POR ROQUETTE PINTO E HENRY MORIZE, TINHAM COMO INTENTO UMA RÁDIO EDUCATIVA E CULTURAL. NA CONTEMPORANEIDADE, AS RÁDIOS ADERIRAM A UM CONTEXTO HIPERTEXTUAL E CONTAM COM COLABORADORES DE DIVERSAS ÁREAS DO CONHECIMENTO PARA AUXILIAR ESTE PROCESSO. DE ACORDO COM ESSE CONTEXTO, VALE LEMBRAR QUE A WEB DISPONIBILIZA, ENTRE OUTROS ASPECTOS, EXPRESSIVO VOLUME DE INFORMAÇÃO, O QUAL, SEM A DEVIDA ORGANIZAÇÃO, INVIABILIZA SUA IDENTIFICAÇÃO, ACESSO E UTILIZAÇÃO. EM DECORRÊNCIA, O ESTUDO EM TELA AO EVIDENCIAR OS DOCUMENTOS SONOROS, APRESENTA COMO FIO CONDUTOR A SEGUINTE QUESTÃO: COMO ESTÃO ORGANIZADOS OS DOCUMENTOS SONOROS NA AMBIENTE WEB DA RÁDIO UEL FM VISANDO SUA RÁPIDA E COMPLETA RECUPERAÇÃO? PARA ORIENTAR A BUSCA POR RESPOSTAS, CONSTITUIU-SE COMO OBJETIVO GERAL, PROPOR RECOMENDAÇÕES PA

In [30]:
len(query.objects)

1